# SciKit Learn Support Vector Machines regression

# Imports

In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Read data

In [3]:
# dataset part 1
dataset = pd.read_csv('../dataset/smog_part1.csv', index_col = 0)

In [4]:
dataset = dataset[['pm25_x', 'pm25_y', 'temperatureSht', 'humiditySht', 'pressure']]
dataset = dataset[dataset['humiditySht'] >= 60]

dataset.head()

,pm25_x,pm25_y,temperatureSht,humiditySht,pressure
0,19.0,13.8,6.1,72,986
1,16.7,14.4,5.9,74,986
2,21.8,15.1,5.7,75,986
3,21.8,18.0,5.7,76,986
4,25.0,19.0,5.5,77,986


# Train and test split

Split data for train and test sets and normalze

In [5]:
Y = dataset['pm25_x'].to_numpy().reshape(-1, 1)
X = dataset['pm25_y'].to_numpy().reshape(-1, 1)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 0)

In [7]:
scaler = preprocessing.StandardScaler().fit(x_train)
y_scaler = max(y_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

y_train = y_train / y_scaler
y_test = y_test / y_scaler

# SKLearn SVR

In [14]:
svr = SVR()

svr.fit(x_train,y_train.ravel())

train_dataset_score = svr.score(x_train,y_train)
test_dataset_score = svr.score(x_test, y_test)

print('R2 on train dataset: ', round(train_dataset_score * 100, 2))
print('R2 on test dataset: ', round(test_dataset_score * 100, 2))

R2 on train dataset:  24.13
R2 on test dataset:  23.4


# HalvingGridSearchCV tuner

In [15]:
param_grid = {
    'C': [0.01, 0.1]+[i for i in range(1, 12)],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'degree': [i for i in range(12)],
    'gamma': ['scale', 'auto'],
    'tol': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
    'epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
    'shrinking': [True, False],
    'cache_size': [2**i for i in range(0, 12, 2)],
}

In [16]:
param_grid = {
    'C': [2],
    'kernel': ['linear'],
    'degree': [1, 2, 4, 8, 16, 32, 64, 128, 256],
    'gamma': ['scale', 'auto'],
    'tol': [0.001],
    'epsilon': [0.001],#[0.01],
    'shrinking': [True]#[False]
}

In [17]:
grid_cv_svr = HalvingGridSearchCV(SVR(),
                           param_grid,
                           cv=5,
                           verbose=1,
                           n_jobs=-1)
grid_cv_svr

HalvingGridSearchCV(estimator=SVR(), n_jobs=-1,
                    param_grid={'C': [2],
                                'degree': [1, 2, 4, 8, 16, 32, 64, 128, 256],
                                'epsilon': [0.001], 'gamma': ['scale', 'auto'],
                                'kernel': ['linear'], 'shrinking': [True],
                                'tol': [0.001]},
                    verbose=1)

In [18]:
%%time
grid_cv_svr.fit(x_train,y_train.ravel())

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 1660
max_resources_: 14946
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 18
n_resources: 1660
Fitting 5 folds for each of 18 candidates, totalling 90 fits
----------
iter: 1
n_candidates: 6
n_resources: 4980
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 2
n_candidates: 2
n_resources: 14940
Fitting 5 folds for each of 2 candidates, totalling 10 fits
CPU times: user 1min 13s, sys: 513 ms, total: 1min 13s
Wall time: 2min 44s


HalvingGridSearchCV(estimator=SVR(), n_jobs=-1,
                    param_grid={'C': [2],
                                'degree': [1, 2, 4, 8, 16, 32, 64, 128, 256],
                                'epsilon': [0.001], 'gamma': ['scale', 'auto'],
                                'kernel': ['linear'], 'shrinking': [True],
                                'tol': [0.001]},
                    verbose=1)

In [19]:
print('Best parameters: ', grid_cv_svr.best_params_)
print('R2: ', round(grid_cv_svr.best_score_ * 100, 2))

Best parameters:  {'C': 2, 'degree': 256, 'epsilon': 0.001, 'gamma': 'scale', 'kernel': 'linear', 'shrinking': True, 'tol': 0.001}
R2:  79.16


In [21]:
best_svr_model = grid_cv_svr.best_estimator_

y_pred = best_svr_model.predict(x_test) * y_scaler

r2 = r2_score(y_test * y_scaler,y_pred) * 100
mse =  mean_squared_error(y_test * y_scaler, y_pred)
mae = mean_absolute_error(y_test * y_scaler, y_pred)

print('R2: ', round(r2, 2))
print('MSE: ',round(mse, 2))
print('MAE: ',round(mae, 2))

R2:  79.93
MSE:  10.51
MAE:  1.54


In [8]:
svr = SVR(kernel='linear',
           tol=0.00001,
           C=16,
           epsilon=0.01,
           shrinking=True)

svr.fit(x_train,y_train.ravel())

y_pred = svr.predict(x_test) * y_scaler

r2 = r2_score(y_test * y_scaler,y_pred) * 100
mse =  mean_squared_error(y_test * y_scaler, y_pred)
mae = mean_absolute_error(y_test * y_scaler, y_pred)

print('R2: ', r2)
print('MSE: ',mse)
print('MAE: ',mae)



R2:  79.97154143038199
MSE:  10.488917453126705
MAE:  1.55220921174563
